<a href="https://colab.research.google.com/github/lweislo/CNResults/blob/master/giro2019results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# STEP 1: CLICK PLAY IN THE UPPER LEFT OF THIS CELL (click in the cell first to see the icon or just shift-enter)
# It will take a little bit to go through loading all the requirements.
# Then go to the next cell and click play - nothing will happen that's obvious there.
import requests
import pandas as pd
from bs4 import BeautifulSoup
!pip install splinter
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from google.colab import files
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from time import sleep
!apt-get update
!apt install chromium-chromedriver

In [0]:
# STEP 2: CLICK PLAY ON THIS CELL. This just loads the program. Then scroll to next cell.

def output_file(header_list, table_list):
    outfile = input("Enter your desired CSV filename: " or "giro_results.csv")
    with open(outfile, 'w') as file:
        for item in range(0, len(header_list)):
            try:
                file.write(f'\n {header_list[item]}\n')
                file.write(table_list[item].to_csv(header=False, index=False, encoding='UTF-8'))
            except:
                pass
    print("Note, the accented characters will get munged if you open directly in Excel.")
    print("Open in TextMate first, remove quotes, and save file in Western Latin-1 format.")
    files.download(outfile)
def init_browser():
    # Mac-specific browser init
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    driver = webdriver.Chrome('chromedriver', options=options)
    return driver
def format_tables(headers, table_list):

    print("Please upload the gironames.csv file.")
    uploaded = files.upload()
    for fn in uploaded.keys():
        macro_file=fn
    macro = pd.read_csv(macro_file, header=None, index_col=False, names=['name','value'])
    macros = macro.set_index('name').to_dict()['value']

    for item in range(0, len(table_list)):
        if "Tempo" in table_list[item]:
            table_list[item] = table_list[item][["Unnamed: 1", "Corridore","Tempo"]]
            table_list[item]["Tempo"] = table_list[item]["Tempo"].str.replace("h ", ":").str.replace("\’ ",":").str.replace("”", "")
        elif "punti" in table_list[item]:
            table_list[item] = table_list[item][["Unnamed: 1", "Corridore","punti"]]
        elif "Team.1" in table_list[item]:
            table_list[item] = table_list[item][["Unnamed: 1", "Corridore"]]

    for item in table_list:
        item['Corridore'] = item['Corridore'].map(macros).fillna(item['Corridore'])

    header_list = []
    for item in headers:
        header_list.append(item.split("| ")[1].split(" (Ufficiale)")[0])
    print("Outputting results")
    output_file(header_list, table_list)
    
def parse_tables(stage_tables):
    headers = []
    table_list = []
    for n in range(1,3):    
        for item in stage_tables[n]:
            try:
                head = item.find('h2').get_text()
                headers.append(head)
                tables = item.find('table')
                table_list.append(pd.read_html(str(tables))[0])
                print(f"Formatting results for {head}")
            except:
                print("No valid results found, sorry")
                pass
    format_tables(headers, table_list)
def scrape_giro(stage):
    browser = init_browser()
    browser.get(stage)
    print("Getting results from Giro")
    sleep(5)
    # Scrape the page
    soup = BeautifulSoup(browser.page_source, 'lxml')
    try:
        stage_tables = soup.find_all('div', class_='tabs-content')
        print("Parsing results")
    except ElementDoesNotExist:
        print("Results not found")
    parse_tables(stage_tables)

def start():
    
    print("Enter http://www.giroditalia.it/it/classifiche/ or to enter a different stage, right click on the stage link on Giro page\
    and paste that URL here")
    stage = input("Enter the URL")
    scrape_giro(stage)

In [0]:
start()
